# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [23]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [24]:
gmaps.configure(api_key= g_key)
cities = "cities.csv"
cities_df = pd.read_csv(cities)

In [3]:
cities_df.head()

,city,country,cloudiness,humidity,latitude,longitude,max_temp,wind_speed
0,hermanus,ZA,0,48,-34.4187,19.2345,78.01,8.99
1,bluff,NZ,90,92,-46.6000,168.3333,43.00,12.62
2,faya,SA,40,25,18.3851,42.4509,75.20,11.50
3,busselton,AU,51,89,-33.6500,115.3333,57.00,7.63
4,zabol,AF,88,34,32.2500,67.2500,61.88,7.16


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
humid_df = cities_df['humidity'].astype(float)
max_humidity = humid_df.max()
location = cities_df[['latitude', 'longitude']]

In [10]:
fig = gmaps.figure()
heat = gmaps.heatmap_layer(location, weights = humid_df, max_intensity = max_humidity, point_radius = 6)
fig.add_layer(heat)
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [13]:
best_temp_df = cities_df.loc[(cities_df['max_temp'] > 75) & (cities_df['max_temp'] < 95), :]
best_temp_df = best_temp_df.dropna()
best_temp_df.head()

,city,country,cloudiness,humidity,latitude,longitude,max_temp,wind_speed
0,hermanus,ZA,0,48,-34.4187,19.2345,78.01,8.99
2,faya,SA,40,25,18.3851,42.4509,75.20,11.50
8,santa isabel do rio negro,BR,97,85,-0.4139,-65.0192,80.87,1.36
11,georgetown,MY,20,70,5.4112,100.3354,84.20,5.75
19,butaritari,KI,100,75,3.0707,172.7902,81.05,6.26


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [34]:
hotels = []
latitude = best_temp_df['latitude']
longitude = best_temp_df['longitude']
for i in range(len(best_temp_df)):
    params = {
        "location": f"{latitude},{longitude}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    jsn = requested.json()
    try:
        hotels.append(jsn['results'][0]['name'])
    except:
        hotels.append("")
best_temp_df["Hotels"] = hotels
best_temp_df = best_temp_df.dropna(how='any')
best_temp_df.head()

,city,country,cloudiness,humidity,latitude,longitude,max_temp,wind_speed,Hotel Name,Hotels
0,hermanus,ZA,0,48,-34.4187,19.2345,78.01,8.99,,
2,faya,SA,40,25,18.3851,42.4509,75.20,11.50,,
8,santa isabel do rio negro,BR,97,85,-0.4139,-65.0192,80.87,1.36,,
11,georgetown,MY,20,70,5.4112,100.3354,84.20,5.75,,
19,butaritari,KI,100,75,3.0707,172.7902,81.05,6.26,,


In [37]:
locations = best_temp_df[['latitude', 'longitude']]

In [35]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in best_temp_df.iterrows()]
locations = best_temp_df[['latitude', 'longitude']]

KeyError: 'City'

In [38]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))